In [1]:
import sys
sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Modules')
sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Viz')
sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/MCPC_Clustering/Codes')

datapath='/Users/chandrasekharmukherjee/Home/Centrality framework/SCRNA_benchmark/'


In [2]:
#Import packages
#imports
import time
import pandas as pd
from collections import Counter
import sknetwork


from sknetwork.ranking import PageRank
from sknetwork.ranking import Betweenness
from sknetwork.ranking import Closeness

import umap as umap

from numba.typed import List
import warnings
from numba import njit
import pynndescent
import numpy as np
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import operator
from sklearn.utils.extmath import randomized_svd
from random import randint
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import scipy
from umap.umap_ import *
import math
from random import randint
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import fashion_mnist
import scanpy
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score, roc_auc_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

import community as community_louvain
from sklearn.cluster import KMeans
from sklearn.metrics import fowlkes_mallows_score

from plots import *
import scipy

import igraph 
import networkx as nx

In [3]:
%load_ext autoreload
%autoreload 2
import importlib

from plots import *


import benchmark as bmarks
import metric as met 
import FlowRank as algo
import datasets as dsets
import simulation as simm
import embedding as embed
import newflow as newalgo
import Relative_centrality_v2 as new_score


met=importlib.reload(met)
newalgo=importlib.reload(newalgo)
algo = importlib.reload(algo)
dsets = importlib.reload(dsets)
simm = importlib.reload(simm)
embed=importlib.reload(embed)
bmarks=importlib.reload(bmarks)
new_score=importlib.reload(new_score)

In [ ]:
datanames = ['Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe','Xin', 'Zhengmix8eq','Tcell-medicine','ALM', 'AMB', 'TM', 'VISP']


In [4]:

name='Baron_Human'
kchoice=20

X = scipy.sparse.load_npz(datapath+name + '/data.npz')
label = np.load(datapath+name+'/labels.npy')
print(name,len(label))

#Log transform+PCA
X.data = np.log1p(X.data)
print("Log transform done")
pca = TruncatedSVD(n_components=50)
PX = pca.fit_transform(X)
n=PX.shape[0]
walk_len_c1=int(np.log2(n))
print(PX.shape)


#Calculte inital KNN accuracy
met.KNN_graph_acc(PX,kchoice,0,label)

#Get the KNN edgelist
edge_list,vlist=embed.dir_KNN_graph(PX,kchoice,0)
print(len(edge_list))

Baron_Human 8569
Log transform done
(8569, 50)
